In [1]:
import pandas as pd
import dostoevsky
import string
import re
import datetime
import time
import warnings
#warnings.filterwarnings('ignore')

Предварительно в командной строке нужно сделать: python3 -m dostoevsky download fasttext-social-network-model

In [2]:
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

In [3]:
tokenizer = RegexTokenizer()
model = FastTextSocialNetworkModel(tokenizer=tokenizer)

In [4]:
df=pd.DataFrame()
df['date_num']=list(range(16071, 19723, 1))
df['date']=pd.date_range('2014-01-01', '2023-12-31')
df=df.set_index('date_num')
df['num_c']=0
df['num_c_with']=0
df['sum_mood']=0
df['sum_mood_with']=0

Рассматриваемый временной промежуток:

00:00:00 01.01.2014 = 1388534400

00:00:00 01.01.2024 = 1704067200

In [5]:
df=pd.DataFrame()
df['date_num']=list(range(16071, 19723, 1))
df['date']=pd.date_range('2014-01-01', '2023-12-31')
df=df.set_index('date_num')
df['num_c']=0
df['num_c_with']=0
df['sum_mood']=0
df['sum_mood_with']=0

In [6]:
def work_with_comments(comments):
    comments=comments[comments['text'].notna()]
    comments=comments.reset_index()
    comments=comments.drop(columns=['index'])
    comments['clean_text']=''
    comments['keyword_i']=0
    comments['mood']=0
    i=0
    for i in range (len(comments)):
        c=0
        s=comments.iloc[i]['text']
        s=str(s)
        s=s.lower()
        for p in string.punctuation:
            if p in s:
                s=s.replace(p, '')
        s=s.replace('\n', ' ')
        comments.loc[i,'clean_text']=s
        c=bool(re.search(r'банк россии|банка россии|банку россии|банком россии|банке росси|цб|центробанк|центральный банк|центрального банка|центральному банку|центральным банком|центральном банке|набиуллин', s))
        comments.loc[i,'keyword_i']=comments['keyword_i'][i]+c
        message=[comments.iloc[i]['text']]
        results = model.predict(message, k=5)
        comments.loc[i,'mood']=results[0]['positive']-results[0]['negative']
    return comments

In [7]:
def make_df_per_day(comments):
    i=0
    for i in range(len(comments)):
        if (comments['comment_time'][i]<1704067200 and comments['comment_time'][i]>1388534400):
            df.loc[comments['comment_time'][i]//86400, 'num_c']=df['num_c'][comments['comment_time'][i]//86400]+1
            df.loc[comments['comment_time'][i]//86400,'sum_mood']=df.loc[comments['comment_time'][i]//86400,'sum_mood']+comments['mood'][i]
            if comments['keyword_i'][i]==1:
                df.loc[comments['comment_time'][i]//86400, 'num_c_with']=df['num_c_with'][comments['comment_time'][i]//86400]+1
                df.loc[comments['comment_time'][i]//86400,'sum_mood_with']=df.loc[comments['comment_time'][i]//86400,'sum_mood_with']+comments['mood'][i]
    return df

# aif_ru

In [8]:
data=pd.read_csv('aif_ru_all_comments.csv')
data=work_with_comments(data)
data

,post_id,text,comment_time,from_id,clean_text,keyword_i,mood
0,1138693,что уже Дальневосточный льготный гектар никому...,1584453896,11298783,что уже дальневосточный льготный гектар никому...,0,-0.277056
1,2236421,"Ввезти ведь,а не вывезти!",1692304548,691425151,ввезти ведьа не вывезти,0,-0.022533
2,2236421,"Не хотят наши власти, чтоб людям жилось хорошо...",1692321591,548839550,не хотят наши власти чтоб людям жилось хорошо ...,0,-0.117484
3,2236421,"Клоун что хотел кого-то подкупать тут, очередн...",1692322774,676449624,клоун что хотел когото подкупать тут очередных...,0,-0.143179
4,2236421,Где-то заработал и решил ввести?----Надо разби...,1692326408,710233104,гдето заработал и решил ввестинадо разбираться...,0,-0.167377
...,...,...,...,...,...,...,...
28089,1073150,"Хотя ответ известен:"" денег нет, но вы держите...",1563785656,308977373,хотя ответ известен денег нет но вы держитесь ...,0,-0.161904
28090,1073150,Очень некорректное сравнение. Нужно не чистый ...,1563786877,170470789,очень некорректное сравнение нужно не чистый д...,0,-0.199606
28091,1073150,Акакий неадекват ?),1563787960,243537564,акакий неадекват,0,-0.125505
28092,1073150,"РФ способна расплатиться с долгами, а США нет...",1563797118,279362575,рф способна расплатиться с долгами а сша нет,0,-0.028024


In [9]:
df=make_df_per_day(data)
df

,date,num_c,num_c_with,sum_mood,sum_mood_with
date_num,,,,,
16071,2014-01-01,0,0,0.000000,0.00000
16072,2014-01-02,0,0,0.000000,0.00000
16073,2014-01-03,0,0,0.000000,0.00000
16074,2014-01-04,0,0,0.000000,0.00000
16075,2014-01-05,0,0,0.000000,0.00000
...,...,...,...,...,...
19718,2023-12-27,158,0,-14.495072,0.00000
19719,2023-12-28,79,1,-10.340452,-0.10855
19720,2023-12-29,82,0,-13.484186,0.00000
